In [347]:
import requests
from bs4 import BeautifulSoup
import http.client
import time
from random import random
import pandas as pd
from datetime import datetime
http.client._MAXHEADERS = 1000 #otherwise scraping didn't work

#### Worfklow
1. Functions:
    - GetSoup - creates a BeautifulSoup object from url
    - LoopThroughPage - go through a supplied page and scrape data from each auction
2. Create an empty dictionary with desired variables
3. Manually enter the main page, make soup out of it, scrape for the number of pages
4. Create urls for all the pages and put them into a list
5. Main loop:
    - Go through the main page
    - Go through subsequent pages waiting a little before each loop in order to avoid getting detected as a bot
6. Load the scraped data into a Pandas Dataframe
7. Save the Dataframe as a csv


    
  

In [348]:
def GetSoup(url):
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    return soup    

In [349]:
#main function, pass soup of the main page to scrape each auction on the page
def LoopThroughPage(page):
    i = 1
    offers = page.find_all('article')
    for o in offers:
        #get a link and make a soup out of it, if 'facebook' then it's the end od the page
        link = o.a.get('href')
        if "facebook" in link: break
        #skip ads
        if "/ad/" in link: break
        time.sleep(random())
        #try shouldn't cause problems but potentially could screw up the dict if an error occurs during loading data into dict
        try:
            single_offer = GetSoup(str(link))

            #find section with parameters for each car and scrape the data
            offer_params = single_offer.find_all('li', {'class': 'offer-params__item'})
            params_dict = {}
            #going through each parameter and creating a dictionary
            for param in offer_params:
                label = param.find('span', {'class':'offer-params__label'}).text.strip()
                value = param.find('div', {'class':'offer-params__value'}).text.strip()
                params_dict[label] = value
            #get the price because it's in a different place
            price_span = single_offer.find('span', {'class':'offer-price__number'})
            price = [p.text.strip() for p in price_span]
            params_dict['Cena'] = price[0]
            #link to the auction
            params_dict['link'] = link
            #parameters for each car may differ, we only want those in car_dict, if doesn't exist enter empty string    
            for x in car_dict:
                try:
                    car_dict[x].append(params_dict[x])
                except:
                    car_dict[x].append('')
        except:
            print(f'Error: {link}')
        #if i == 1: break #testing purposes

In [350]:
#Dictionary with all the cars
car_dict = {
'link':[],
'Oferta od':[],
'Kategoria':[],
'Generacja':[],
'Rok produkcji':[],
'Przebieg':[],
'Pojemność skokowa':[],
'Rodzaj paliwa':[],
'Moc':[],
'Skrzynia biegów':[],
'Napęd':[],
'Spalanie W Mieście':[],
'Typ nadwozia':[],
'Liczba drzwi':[],
'Liczba miejsc':[],
'Kolor':[],
'Kraj pochodzenia':[],
'Zarejestrowany w Polsce':[],
'Bezwypadkowy':[],
'Stan':[],
'Cena':[] 
}

In [351]:
url = str('https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&search%5Badvanced_search_expanded%5D=true')
#url = str('https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=2&search%5Badvanced_search_expanded%5D=true')
#soup = BeautifulSoup(requests.get(url).content, 'html.parser')
main_page = GetSoup(url)
no_of_pages = main_page.find_all('a', {'class': 'ooa-xdlax9 eesa4ha0'})[3].text

In [352]:
page_url = []
for i in range(2, int(no_of_pages)+1):
    page_url.append(str(f'https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page={i}&search%5Badvanced_search_expanded%5D=true'))

In [353]:
LoopThroughPage(main_page)
i = 1
for url in page_url:
    time.sleep(random())
    
    #get soup of url
    soup = GetSoup(url)
    LoopThroughPage(soup)
    #if i ==1 : break #testing purposes
    print(url)

https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=2&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=3&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=4&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=5&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=6&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=7&search%5Badvanced_search_expanded%

https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=50&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=51&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=52&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=53&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=54&search%5Badvanced_search_expanded%5D=true
https://www.otomoto.pl/osobowe/volkswagen/golf?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_no_accident%5D=1&page=55&search%5Badvanced_search_exp

In [355]:
df_cars = pd.DataFrame(car_dict)
df_cars

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,...,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
0,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2015,112 000 km,1 395 cm3,Benzyna,125 KM,Manualna,...,6 l/100km,Kompakt,5,5,Szary,Polska,Tak,Tak,Używane,56 400
1,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,III (1991-1998),1997,230 000 km,1 781 cm3,Benzyna,75 KM,Manualna,...,11 l/100km,Kompakt,2,5,Czarny,,Tak,Tak,Używane,3 500
2,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,V (2003-2009),2008,269 000 km,1 390 cm3,Benzyna,122 KM,Manualna,...,,Kombi,5,5,Szary,Holandia,,Tak,Używane,16 500
3,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2022,,1 984 cm3,Benzyna,,Automatyczna,...,,Kompakt,,,Biały,Polska,,Tak,Nowe,249 000
4,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2022,,1 984 cm3,Benzyna,,Automatyczna,...,,Kompakt,,,Czarny,Polska,,Tak,Nowe,251 700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1789,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2015,52 109 km,1 395 cm3,Benzyna,125 KM,Automatyczna,...,6 l/100km,Kompakt,5,5,Granatowy,Polska,Tak,Tak,Używane,66 800
1790,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2015,209 600 km,1 598 cm3,Diesel,110 KM,Manualna,...,3 l/100km,Kombi,5,5,Granatowy,Polska,Tak,Tak,Używane,39 900
1791,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2019,58 000 km,1 498 cm3,Benzyna,150 KM,Automatyczna,...,"6,10 l/100km",Kompakt,5,5,Szary,,Tak,Tak,Używane,85 500
1792,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,V (2003-2009),2005,195 670 km,1 595 cm3,Benzyna,102 KM,Manualna,...,9 l/100km,Kompakt,5,5,Czarny,Niemcy,,Tak,Używane,17 000


In [356]:
today = datetime.today().strftime('%Y-%m-%d')
file_name = f'otomoto_scraping_{today}.csv'
df_cars.to_csv(file_name, index=False)